In [2]:
pip install gensim


  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pprint
import pandas as pd
import gensim
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import *
import numpy as np
np.random.seed(2018)
import nltk
import nltk.stem as stemmer
stemmer = PorterStemmer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [72]:
File = pd.ExcelFile('DatosWoS.xlsx')
df=File.parse('Sheet1')



In [73]:
df.describe()

,referencias
count,78791.000000
mean,45.749616
std,31.669358
min,0.000000
25%,28.000000
50%,39.000000
75%,55.000000
max,724.000000


In [74]:
titles = df["Titulo"]

In [75]:
text_corpus = titles.values.tolist()


In [78]:
print(text_corpus[0])

human versus  


In [77]:
for i in range(len(text_corpus)):
    text_corpus[i] = str(text_corpus[i])
    text_corpus[i] = re.sub('[!,*)@#%(&$_?.^:-]', '', text_corpus[i])
    text_corpus[i] = text_corpus[i].replace('"','')
    text_corpus[i] = text_corpus[i].lower()
    text_corpus[i] = text_corpus[i].replace("artificial","")
    text_corpus[i] = text_corpus[i].replace("intelligence","")
print(text_corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [79]:
indexs = []
for i in range(len(text_corpus)):
    indexs.append(i)

In [80]:
df2 = pd.DataFrame(list(zip(text_corpus, indexs)),
               columns =['Titulo', 'index'])

In [82]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [83]:
documents = df2
doc_sample = documents[documents['index'] == 0].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['human', 'versus', '', '']


 tokenized and lemmatized document: 
['human', 'versu']


In [84]:
processed_docs = documents['Titulo'].map(preprocess)
processed_docs[:10]

0                                   [human, versu]
1                                          [brain]
2            [test, case, studi, intellig, vehicl]
3                              [recommend, system]
4                           [intellectu, properti]
5                                        [spiritu]
6                        [emot, employe, perspect]
7    [safe, futur, discuss, machin, ethic, safeti]
8                  [surpass, human, factual, hoax]
9                          [reflect, ethic, human]
Name: Titulo, dtype: object

In [85]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 human
1 versu
2 brain
3 case
4 intellig
5 studi
6 test
7 vehicl
8 recommend
9 system
10 intellectu


In [86]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [87]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(168, 1), (236, 1), (268, 1), (393, 1), (842, 1), (1275, 1), (1343, 1)]

In [88]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 168 ("model") appears 1 time.
Word 236 ("imag") appears 1 time.
Word 268 ("covid") appears 1 time.
Word 393 ("detect") appears 1 time.
Word 842 ("interpret") appears 1 time.
Word 1275 ("radiographi") appears 1 time.
Word 1343 ("chest") appears 1 time.


In [89]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5499713538886792), (1, 0.8351835186962523)]


In [90]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [91]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.027*"optim" + 0.027*"model" + 0.017*"support" + 0.017*"comput" + 0.016*"algorithm" + 0.016*"decis" + 0.015*"predict" + 0.013*"base" + 0.012*"applic" + 0.011*"nonlinear"
Topic: 1 
Words: 0.026*"studi" + 0.017*"data" + 0.014*"analysi" + 0.012*"health" + 0.012*"learn" + 0.012*"survey" + 0.011*"servic" + 0.011*"case" + 0.010*"technolog" + 0.010*"social"
Topic: 2 
Words: 0.043*"learn" + 0.030*"predict" + 0.026*"machin" + 0.019*"cancer" + 0.015*"deep" + 0.015*"patient" + 0.014*"studi" + 0.014*"featur" + 0.013*"diseas" + 0.012*"model"
Topic: 3 
Words: 0.026*"problem" + 0.019*"represent" + 0.018*"graph" + 0.013*"solv" + 0.012*"knowledg" + 0.010*"spars" + 0.010*"base" + 0.009*"model" + 0.008*"human" + 0.007*"robot"
Topic: 4 
Words: 0.027*"recognit" + 0.020*"activ" + 0.018*"base" + 0.016*"imag" + 0.015*"model" + 0.011*"attent" + 0.011*"human" + 0.008*"learningbas" + 0.008*"multiscal" + 0.008*"speech"
Topic: 5 
Words: 0.055*"learn" + 0.019*"model" + 0.017*"process" + 0.016*"pla

In [92]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"optim" + 0.009*"network" + 0.007*"algorithm" + 0.007*"base" + 0.007*"imag" + 0.007*"classif" + 0.006*"model" + 0.006*"analysi" + 0.006*"learn" + 0.006*"neural"
Topic: 1 Word: 0.010*"network" + 0.009*"segment" + 0.008*"imag" + 0.008*"learn" + 0.007*"graph" + 0.007*"model" + 0.007*"neural" + 0.007*"semant" + 0.006*"deep" + 0.006*"base"
Topic: 2 Word: 0.008*"learn" + 0.007*"base" + 0.007*"algorithm" + 0.007*"robot" + 0.007*"problem" + 0.007*"model" + 0.006*"optim" + 0.006*"time" + 0.005*"smart" + 0.005*"seri"
Topic: 3 Word: 0.008*"learn" + 0.007*"model" + 0.006*"base" + 0.006*"network" + 0.006*"internet" + 0.006*"express" + 0.006*"recognit" + 0.005*"approach" + 0.005*"data" + 0.005*"thing"
Topic: 4 Word: 0.008*"model" + 0.007*"predict" + 0.006*"base" + 0.006*"optim" + 0.006*"learn" + 0.006*"network" + 0.005*"studi" + 0.005*"analysi" + 0.005*"energi" + 0.005*"neural"
Topic: 5 Word: 0.011*"imag" + 0.009*"base" + 0.009*"learn" + 0.008*"model" + 0.008*"network" + 0.007*"

In [93]:
processed_docs[4310]

['aidcov',
 'interpret',
 'model',
 'detect',
 'covid',
 'chest',
 'radiographi',
 'imag']

In [94]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.443479061126709	 
Topic: 0.038*"imag" + 0.033*"algorithm" + 0.029*"base" + 0.024*"learn" + 0.023*"detect" + 0.019*"classif" + 0.017*"deep" + 0.017*"featur" + 0.013*"cluster" + 0.012*"improv"

Score: 0.3001474440097809	 
Topic: 0.039*"system" + 0.038*"control" + 0.021*"base" + 0.016*"design" + 0.016*"optim" + 0.014*"distribut" + 0.012*"multiag" + 0.010*"particl" + 0.010*"algorithm" + 0.010*"swarm"

Score: 0.1688171625137329	 
Topic: 0.026*"studi" + 0.017*"data" + 0.014*"analysi" + 0.012*"health" + 0.012*"learn" + 0.012*"survey" + 0.011*"servic" + 0.011*"case" + 0.010*"technolog" + 0.010*"social"

Score: 0.012511170469224453	 
Topic: 0.043*"learn" + 0.030*"predict" + 0.026*"machin" + 0.019*"cancer" + 0.015*"deep" + 0.015*"patient" + 0.014*"studi" + 0.014*"featur" + 0.013*"diseas" + 0.012*"model"

Score: 0.012509666383266449	 
Topic: 0.104*"network" + 0.052*"neural" + 0.029*"base" + 0.027*"detect" + 0.019*"convolut" + 0.019*"deep" + 0.017*"model" + 0.014*"learn" + 0.010*"classif

In [95]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8874461054801941	 
Topic: 0.009*"detect" + 0.009*"base" + 0.009*"network" + 0.008*"learn" + 0.008*"imag" + 0.008*"deep" + 0.007*"algorithm" + 0.006*"recognit" + 0.006*"optim" + 0.006*"neural"

Score: 0.012508191168308258	 
Topic: 0.008*"studi" + 0.007*"learn" + 0.007*"genom" + 0.007*"patient" + 0.007*"analysi" + 0.006*"clinic" + 0.006*"machin" + 0.006*"model" + 0.006*"diseas" + 0.005*"human"

Score: 0.012506874278187752	 
Topic: 0.008*"model" + 0.007*"predict" + 0.006*"base" + 0.006*"optim" + 0.006*"learn" + 0.006*"network" + 0.005*"studi" + 0.005*"analysi" + 0.005*"energi" + 0.005*"neural"

Score: 0.012506592087447643	 
Topic: 0.010*"network" + 0.009*"segment" + 0.008*"imag" + 0.008*"learn" + 0.007*"graph" + 0.007*"model" + 0.007*"neural" + 0.007*"semant" + 0.006*"deep" + 0.006*"base"

Score: 0.012506299652159214	 
Topic: 0.011*"imag" + 0.009*"base" + 0.009*"learn" + 0.008*"model" + 0.008*"network" + 0.007*"data" + 0.007*"classif" + 0.006*"algorithm" + 0.006*"deep" + 0.006*"

In [96]:
unseen_document = 'Linguistic Query Answering on Data Cubes with Time Dimension'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.49018365144729614	 Topic: 0.027*"data" + 0.018*"analysi" + 0.014*"base" + 0.014*"method" + 0.012*"imag"
Score: 0.395486056804657	 Topic: 0.026*"problem" + 0.019*"represent" + 0.018*"graph" + 0.013*"solv" + 0.012*"knowledg"
Score: 0.014299462549388409	 Topic: 0.026*"studi" + 0.017*"data" + 0.014*"analysi" + 0.012*"health" + 0.012*"learn"
Score: 0.014295801520347595	 Topic: 0.104*"network" + 0.052*"neural" + 0.029*"base" + 0.027*"detect" + 0.019*"convolut"
Score: 0.01429023128002882	 Topic: 0.038*"imag" + 0.033*"algorithm" + 0.029*"base" + 0.024*"learn" + 0.023*"detect"
Score: 0.014289796352386475	 Topic: 0.027*"optim" + 0.027*"model" + 0.017*"support" + 0.017*"comput" + 0.016*"algorithm"
Score: 0.01428909506648779	 Topic: 0.043*"learn" + 0.030*"predict" + 0.026*"machin" + 0.019*"cancer" + 0.015*"deep"
Score: 0.014288748614490032	 Topic: 0.055*"learn" + 0.019*"model" + 0.017*"process" + 0.016*"plan" + 0.015*"base"
Score: 0.014288614504039288	 Topic: 0.039*"system" + 0.038*"contr